In [ ]:
from lib.imports import *

In [ ]:
skip_idx = [19,24,26,32,34,38,40,45,52,55,70]
train_idx = range(71)
window_size = 101
X,y = load_and_window_nursing_list(train_idx,window_size=window_size)

In [ ]:
X_train,X_dev,y_train,y_dev = train_test_split(X,y,test_size=.2,stratify=y)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=64,shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=64,shuffle=True)

In [ ]:
from torch import nn
device = 'cuda'
model = MLP(window_size=window_size).to(device=device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

In [ ]:
loss_tr = []
loss_dev = []
pbar = tqdm(range(10))

for epoch in pbar:
    # train loop
    model.train()
    loss_tr_total = 0
    for (X_tr,y_tr) in trainloader:
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))

    # dev loop
    model.eval()
    loss_dev_total = 0
    for (X_dv,y_dv) in devloader:
        X_dv,y_dv = X_dv.to(device),y_dv.to(device)
        logits = model(X_dv)
        loss = criterion(logits,y_dv)
        loss_dev_total += loss.item()
    loss_dev.append(loss_dev_total/len(devloader))
plt.plot(loss_tr,label='train')
plt.plot(loss_dev,label='dev')
plt.legend()

In [ ]:
test_evaluation(devloader,model,criterion)

In [ ]:
from sklearn.metrics import f1_score
f1i = []
for idx in tqdm(range(71)):
    if(idx in skip_idx or idx in train_idx):
        continue
    X,y = load_and_window_nursing_list([idx])
    y_pred = torch.sigmoid(model(X.cuda())).round().detach().cpu()
    f1i.append(f1_score(y_true=y,y_pred=y_pred,average='macro'))

sns.kdeplot(f1i,bw_adjust=.4)
print(torch.tensor(f1i).mean()) #.6653
print(torch.tensor(f1i).std()) #.1244